In [1]:
curated_states_partition = catalog.load("center_out_curated_states_pkl")
spectrogram_dict = catalog.load("center_out_spectrogram_std_pkl")

                    INFO     Loading data from 'center_out_curated_states_pkl'                  ]8;id=772999;file:///home/sshah/.conda/envs/bci-pipeline/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=281021;file:///home/sshah/.conda/envs/bci-pipeline/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             (PartitionedDataSet)...                                                               

                    INFO     Loading data from 'center_out_spectrogram_std_pkl'                 ]8;id=674472;file:///home/sshah/.conda/envs/bci-pipeline/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=970555;file:///home/sshah/.conda/envs/bci-pipeline/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             (PartitionedDataSet)...                                                               

In [2]:
curated_states_dict = curated_states_partition['ccCenterOut_20221027_S03']()
sxx_data_dict = spectrogram_dict['ccCenterOut_20221027_S03']()

In [3]:
from scripts.utility_scripts import create_closure, create_closure_func

import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

In [11]:
state = 1
session_details_dict = curated_states_dict[state]

model_data_params = context.params['model_data_params']

pre_stimulus_time = model_data_params['pre_stimulus_time']
post_completion_time = model_data_params['post_completion_time']

window_size = model_data_params['window_size']
shift = model_data_params['shift']

current_experiment = context.params['current_experiment']
sessions = context.params['sessions']
patient_id = context.params['patient_id']

def _window_spectrogram(data_func, indices, shift, window_size):
    sxx_data_dict = data_func()
    
    sxx = sxx_data_dict['sxx']
    
    return np.moveaxis(sliding_window_view(sxx[:, indices, :][:, ::shift, :], window_shape=window_size, axis=1), [0, -1], [1, -2])
    

def generate_model_windowed_sxx_data(spectrogram_dict, curated_states_partition, sessions, model_data_params, current_experiment, patient_id):
    pre_stimulus_time = model_data_params['pre_stimulus_time']
    post_completion_time = model_data_params['post_completion_time']

    window_size = model_data_params['window_size']
    shift = model_data_params['shift']

    model_data_dict = {}
    model_data_filenames_dict = {}
    global_trial_idx = 0
    for session_type, session_data in sessions[patient_id][current_experiment].items():
        for sxx_partition_key, sxx_partition_func in spectrogram_dict.items():
            date = sxx_partition_key.split('_')[-2]
            session = sxx_partition_key.split('_')[-1]

            if date in session_data.keys() and session in session_data[date]:
                curated_states_data_func = curated_states_partition[sxx_partition_key]
                curated_states_dict = curated_states_data_func()
                # sxx_data_dict = sxx_partition_func()

                # sampling_rate = sxx_data_dict['sampling_rate']
#                 sampling_rate = 1000

                model_data_metadata = {}
                for state, state_information in curated_states_dict.items():
                    cur_dict = {}

                    start_end_idx = state_information['start_end_idx']
                    
                    sampling_rate = state_information['sampling_rate']
                    
                    samples_pre = int(np.ceil((pre_stimulus_time * sampling_rate)/shift))
                    samples_post = int(np.ceil((post_completion_time * sampling_rate)/shift)) 
                    
                    cur_dict['start_end_idx'] = [(x[0]-samples_pre, x[1]+samples_post) for x in start_end_idx]
                    cur_dict['unique_val_idx'] = [np.arange(x[0], x[1]+1) for x in cur_dict['start_end_idx']]
                    cur_dict['num_steps'] = [len(x) for x in cur_dict['unique_val_idx']]

                    model_data_metadata[state] = cur_dict

                # sxx = sxx_data_dict['sxx']

                intermed_model_dict = {}
                for state, metadata_dict in model_data_metadata.items():
                    if state == 0:
                        continue
                        
                    trial_idx_list = []
                    local_trial_idx_list = []
                    for trial_idx, indices in enumerate(metadata_dict['unique_val_idx']):
                        model_data_dict[f"{session_type}_{date}_{session}_T{trial_idx}_state{state}"] = create_closure_func(_window_spectrogram, sxx_partition_func, indices, shift, window_size)
                        
                        trial_idx_list.append(global_trial_idx)
                        global_trial_idx += 1
                        
                        local_trial_idx_list.append(trial_idx)
                        
                    cur_partition_dict = model_data_filenames_dict.setdefault(sxx_partition_key, {})

                    cur_partition_dict[state] = {
                            'date': date,
                            'session_type': session_type,
                            'session': session,
                            'state': state,
                            'local_trials_idx_list': local_trial_idx_list,
                            'global_trials_idx_list': trial_idx_list
                    }
                        
    return model_data_dict, model_data_filenames_dict

In [84]:
min_max_list = []
for sxx_partition_key, sxx_partition_func in spectrogram_dict.items():
    sxx_data_dict = sxx_partition_func()
    
    sxx = sxx_data_dict['sxx']
    
    min_max_list.append((np.min(sxx), np.max(sxx)))

In [85]:
min_max_list

[
    (-13.990148768566117, 6.374211011876654),
    (-12.968163208659362, 5.937483234596849),
    (-12.653183634942268, 5.696542451898437),
    (-12.2995071902545, 5.478741180956943),
    (-12.696109581137966, 6.34625672707988),
    (-13.052663090011837, 7.405774786883838),
    (-15.23578992869043, 7.035384824443133),
    (-14.65271508679479, 5.346074616076721),
    (-14.223946292105063, 4.255882959776221),
    (-13.514993142699895, 4.262524239006428),
    (-13.414841166696991, 5.2733662535839025),
    (-13.804207320672575, 4.762242689613853),
    (-14.219154972028335, 4.076851923928893),
    (-35.70550584570492, 12.203497980269796),
    (-14.707335435141744, 9.050254763890178),
    (-13.877185848135031, 10.379559729179432),
    (-15.8934965590094, 11.029405261533533),
    (-14.349913345098205, 5.829274160618642),
    (-16.090753951319932, 4.239026443277557),
    (-13.823241526668928, 4.072786387564452),
    (-77.65828331327475, 12.213827678125236),
    (-12.584338312023402, 5.440163320022357),
    (-13.296949275985877, 3.871171094311055),
    (-13.641431395644029, 5.503054863588154),
    (-13.943792181850124, 3.906498091783551),
    (-13.88025400235313, 4.294512920130019),
    (-12.961204106544201, 3.794883544603795),
    (-15.054118181602636, 6.650892182281899)
]

In [12]:
model_data_dict, model_data_filenames_dict = generate_model_windowed_sxx_data(spectrogram_dict, curated_states_partition, sessions, model_data_params, current_experiment, patient_id)

In [88]:

model_data_dict
    

{
    'overt_20221027_S03_T0_state1': <function create_closure_func.<locals>.closure at 0x7f1f75ad80d0>,
    'overt_20221027_S03_T1_state1': <function create_closure_func.<locals>.closure at 0x7f1f75ad8790>,
    'overt_20221027_S03_T2_state1': <function create_closure_func.<locals>.closure at 0x7f1f76aa1c60>,
    'overt_20221027_S03_T3_state1': <function create_closure_func.<locals>.closure at 0x7f1f76aa1a20>,
    'overt_20221027_S03_T4_state1': <function create_closure_func.<locals>.closure at 0x7f1f76aa1990>,
    'overt_20221027_S03_T0_state2': <function create_closure_func.<locals>.closure at 0x7f1f76aa1bd0>,
    'overt_20221027_S03_T1_state2': <function create_closure_func.<locals>.closure at 0x7f1f76aa16c0>,
    'overt_20221027_S03_T2_state2': <function create_closure_func.<locals>.closure at 0x7f1f76aa1630>,
    'overt_20221027_S03_T3_state2': <function create_closure_func.<locals>.closure at 0x7f1f76aa15a0>,
    'overt_20221027_S03_T4_state2': <function create_closure_func.<locals>.closure at 0x7f1f76aa1510>,
    'overt_20221027_S03_T0_state3': <function create_closure_func.<locals>.closure at 0x7f1f76aa1480>,
    'overt_20221027_S03_T1_state3': <function create_closure_func.<locals>.closure at 0x7f1f76aa13f0>,
    'overt_20221027_S03_T2_state3': <function create_closure_func.<locals>.closure at 0x7f1f76aa1360>,
    'overt_20221027_S03_T3_state3': <function create_closure_func.<locals>.closure at 0x7f1f76aa12d0>,
    'overt_20221027_S03_T4_state3': <function create_closure_func.<locals>.closure at 0x7f1f76aa1240>,
    'overt_20221027_S03_T0_state4': <function create_closure_func.<locals>.closure at 0x7f1f76aa11b0>,
    'overt_20221027_S03_T1_state4': <function create_closure_func.<locals>.closure at 0x7f1f76aa1120>,
    'overt_20221027_S03_T2_state4': <function create_closure_func.<locals>.closure at 0x7f1f76aa1090>,
    'overt_20221027_S03_T3_state4': <function create_closure_func.<locals>.closure at 0x7f1f76aa1000>,
    'overt_20221027_S03_T4_state4': <function create_closure_func.<locals>.closure at 0x7f1f76aa0f70>,
    'overt_20221027_S03_T0_state5': <function create_closure_func.<locals>.closure at 0x7f1f76aa0ee0>,
    'overt_20221027_S03_T1_state5': <function create_closure_func.<locals>.closure at 0x7f1f76aa0e50>,
    'overt_20221027_S03_T2_state5': <function create_closure_func.<locals>.closure at 0x7f1f76aa0dc0>,
    'overt_20221027_S03_T3_state5': <function create_closure_func.<locals>.closure at 0x7f1f76aa0d30>,
    'overt_20221027_S03_T4_state5': <function create_closure_func.<locals>.closure at 0x7f1f76aa0ca0>,
    'overt_20221027_S03_T0_state6': <function create_closure_func.<locals>.closure at 0x7f1f76aa0c10>,
    'overt_20221027_S03_T1_state6': <function create_closure_func.<locals>.closure at 0x7f1f76aa0b80>,
    'overt_20221027_S03_T2_state6': <function create_closure_func.<locals>.closure at 0x7f1f76aa0af0>,
    'overt_20221027_S03_T3_state6': <function create_closure_func.<locals>.closure at 0x7f1f76aa0a60>,
    'overt_20221027_S03_T4_state6': <function create_closure_func.<locals>.closure at 0x7f1f76aa09d0>,
    'overt_20221027_S03_T0_state7': <function create_closure_func.<locals>.closure at 0x7f1f76aa0940>,
    'overt_20221027_S03_T1_state7': <function create_closure_func.<locals>.closure at 0x7f1f76aa08b0>,
    'overt_20221027_S03_T2_state7': <function create_closure_func.<locals>.closure at 0x7f1f76aa0790>,
    'overt_20221027_S03_T3_state7': <function create_closure_func.<locals>.closure at 0x7f1f76aa0700>,
    'overt_20221027_S03_T4_state7': <function create_closure_func.<locals>.closure at 0x7f1f76aa0670>,
    'overt_20221027_S03_T0_state8': <function create_closure_func.<locals>.closure at 0x7f1f76aa05e0>,
    'overt_20221027_S03_T1_state8': <function create_closure_func.<locals>.closure at 0x7f1f76aa0550>,
    'overt_20221027_S03_T2_state8': <function create_closure_func.<locals>.closure at 0x7f1f76aa04c0>,
    'overt_20221027_S03_T3_state8': <function create_closure_func.<locals>.closure a

In [56]:
np.min(data)

-12.138117659240523

In [86]:
data_small = data.astype(np.float16)

In [82]:
x = np.array([32790])
x = x.astype(np.float16)
# x = x + 1
x

array([32800.], dtype=float16)

In [65]:
data_small.dtype

dtype('float16')

In [87]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

import h5py


# np.save("mydata.npy", data)
np.save("mydata2.npy", data_small)

# with h5py.File("mytestfile.hdf5", "w") as f:
#     dset = f.create_dataset("dataset", [data], dtype='float32')



# x = np.array([0, 1, 2])

# pa_table = pa.table({"data": data})

# pq.write_table(pa_table, "ndarray.parquet")


In [14]:
model_data_dict

{
    'overt_20221027_S03_T0_state1': <function create_closure_func.<locals>.closure at 0x7fecf184bd90>,
    'overt_20221027_S03_T1_state1': <function create_closure_func.<locals>.closure at 0x7fecf18adbd0>,
    'overt_20221027_S03_T2_state1': <function create_closure_func.<locals>.closure at 0x7fecf0f63be0>,
    'overt_20221027_S03_T3_state1': <function create_closure_func.<locals>.closure at 0x7fecf0f63c70>,
    'overt_20221027_S03_T4_state1': <function create_closure_func.<locals>.closure at 0x7fecf0f63e20>,
    'overt_20221027_S03_T0_state2': <function create_closure_func.<locals>.closure at 0x7fecf0f63a30>,
    'overt_20221027_S03_T1_state2': <function create_closure_func.<locals>.closure at 0x7fecf13d43a0>,
    'overt_20221027_S03_T2_state2': <function create_closure_func.<locals>.closure at 0x7fecf13d4430>,
    'overt_20221027_S03_T3_state2': <function create_closure_func.<locals>.closure at 0x7fecf13d44c0>,
    'overt_20221027_S03_T4_state2': <function create_closure_func.<locals>.closure at 0x7fecf13d4550>,
    'overt_20221027_S03_T0_state3': <function create_closure_func.<locals>.closure at 0x7fecf13d45e0>,
    'overt_20221027_S03_T1_state3': <function create_closure_func.<locals>.closure at 0x7fecf13d4670>,
    'overt_20221027_S03_T2_state3': <function create_closure_func.<locals>.closure at 0x7fecf13d4700>,
    'overt_20221027_S03_T3_state3': <function create_closure_func.<locals>.closure at 0x7fecf13d4790>,
    'overt_20221027_S03_T4_state3': <function create_closure_func.<locals>.closure at 0x7fecf13d4820>,
    'overt_20221027_S03_T0_state4': <function create_closure_func.<locals>.closure at 0x7fecf13d48b0>,
    'overt_20221027_S03_T1_state4': <function create_closure_func.<locals>.closure at 0x7fecf13d4940>,
    'overt_20221027_S03_T2_state4': <function create_closure_func.<locals>.closure at 0x7fecf13d49d0>,
    'overt_20221027_S03_T3_state4': <function create_closure_func.<locals>.closure at 0x7fecf13d4a60>,
    'overt_20221027_S03_T4_state4': <function create_closure_func.<locals>.closure at 0x7fecf13d4af0>,
    'overt_20221027_S03_T0_state5': <function create_closure_func.<locals>.closure at 0x7fecf13d4b80>,
    'overt_20221027_S03_T1_state5': <function create_closure_func.<locals>.closure at 0x7fecf13d4c10>,
    'overt_20221027_S03_T2_state5': <function create_closure_func.<locals>.closure at 0x7fecf13d4ca0>,
    'overt_20221027_S03_T3_state5': <function create_closure_func.<locals>.closure at 0x7fecf13d4d30>,
    'overt_20221027_S03_T4_state5': <function create_closure_func.<locals>.closure at 0x7fecf13d4dc0>,
    'overt_20221027_S03_T0_state6': <function create_closure_func.<locals>.closure at 0x7fecf13d4e50>,
    'overt_20221027_S03_T1_state6': <function create_closure_func.<locals>.closure at 0x7fecf13d4ee0>,
    'overt_20221027_S03_T2_state6': <function create_closure_func.<locals>.closure at 0x7fecf13d4f70>,
    'overt_20221027_S03_T3_state6': <function create_closure_func.<locals>.closure at 0x7fecf13d5000>,
    'overt_20221027_S03_T4_state6': <function create_closure_func.<locals>.closure at 0x7fecf13d5090>,
    'overt_20221027_S03_T0_state7': <function create_closure_func.<locals>.closure at 0x7fecf13d5120>,
    'overt_20221027_S03_T1_state7': <function create_closure_func.<locals>.closure at 0x7fecf13d51b0>,
    'overt_20221027_S03_T2_state7': <function create_closure_func.<locals>.closure at 0x7fecf13d5240>,
    'overt_20221027_S03_T3_state7': <function create_closure_func.<locals>.closure at 0x7fecf13d52d0>,
    'overt_20221027_S03_T4_state7': <function create_closure_func.<locals>.closure at 0x7fecf13d5360>,
    'overt_20221027_S03_T0_state8': <function create_closure_func.<locals>.closure at 0x7fecf13d53f0>,
    'overt_20221027_S03_T1_state8': <function create_closure_func.<locals>.closure at 0x7fecf13d5480>,
    'overt_20221027_S03_T2_state8': <function create_closure_func.<locals>.closure at 0x7fecf13d5510>,
    'overt_20221027_S03_T3_state8': <function create_closure_func.<locals>.closure a

In [7]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

test_spectrogram_dict = np.array([[[x] for x in range(50)]])

shift = 2
indices = np.array(range(50))
window_size = 10

np.moveaxis(sliding_window_view(test_spectrogram_dict[:, indices, :][:, ::shift, :], window_shape=window_size, axis=1), [0, -1], [1, -2])

array([[[[ 0],
         [ 2],
         [ 4],
         [ 6],
         [ 8],
         [10],
         [12],
         [14],
         [16],
         [18]]],


       [[[ 2],
         [ 4],
         [ 6],
         [ 8],
         [10],
         [12],
         [14],
         [16],
         [18],
         [20]]],


       [[[ 4],
         [ 6],
         [ 8],
         [10],
         [12],
         [14],
         [16],
         [18],
         [20],
         [22]]],


       [[[ 6],
         [ 8],
         [10],
         [12],
         [14],
         [16],
         [18],
         [20],
         [22],
         [24]]],


       [[[ 8],
         [10],
         [12],
         [14],
         [16],
         [18],
         [20],
         [22],
         [24],
         [26]]],


       [[[10],
         [12],
         [14],
         [16],
         [18],
         [20],
         [22],
         [24],
         [26],
         [28]]],


       [[[12],
         [14],
         [16],
         [18],
         [20],
         [22],
         [24],
         [26],
         [28],
         [30]]],


       [[[14],
         [16],
         [18],
         [20],
         [22],
         [24],
         [26],
         [28],
         [30],
         [32]]],


       [[[16],
         [18],
         [20],
         [22],
         [24],
         [26],
         [28],
         [30],
         [32],
         [34]]],


       [[[18],
         [20],
         [22],
         [24],
         [26],
         [28],
         [30],
         [32],
         [34],
         [36]]],


       [[[20],
         [22],
         [24],
         [26],
         [28],
         [30],
         [32],
         [34],
         [36],
         [38]]],


       [[[22],
         [24],
         [26],
         [28],
         [30],
         [32],
         [34],
         [36],
         [38],
         [40]]],


       [[[24],
         [26],
         [28],
         [30],
         [32],
         [34],
         [36],
         [38],
         [40],
         [42]]],


       [[[26],
         [28],
         [30],
         [32],
         [34],
         [36],
         [38],
         [40],
         [42],
         [44]]],


       [[[28],
         [30],
         [32],
         [34],
         [36],
         [38],
         [40],
         [42],
         [44],
         [46]]],


       [[[30],
         [32],
         [34],
         [36],
         [38],
         [40],
         [42],
         [44],
         [46],
         [48]]]])

In [13]:
model_data_filenames_dict

{
    'ccCenterOut_20221027_S03': {
        1: {
            'date': '20221027',
            'session_type': 'overt',
            'session': 'S03',
            'state': 1,
            'local_trials_idx_list': [0, 1, 2, 3, 4],
            'global_trials_idx_list': [0, 1, 2, 3, 4]
        },
        2: {
            'date': '20221027',
            'session_type': 'overt',
            'session': 'S03',
            'state': 2,
            'local_trials_idx_list': [0, 1, 2, 3, 4],
            'global_trials_idx_list': [5, 6, 7, 8, 9]
        },
        3: {
            'date': '20221027',
            'session_type': 'overt',
            'session': 'S03',
            'state': 3,
            'local_trials_idx_list': [0, 1, 2, 3, 4],
            'global_trials_idx_list': [10, 11, 12, 13, 14]
        },
        4: {
            'date': '20221027',
            'session_type': 'overt',
            'session': 'S03',
            'state': 4,
            'local_trials_idx_list': [0, 1, 2, 3, 4],
            'global_trials_idx_list': [15, 16, 17, 18, 19]
        },
        5: {
            'date': '20221027',
            'session_type': 'overt',
            'session': 'S03',
            'state': 5,
            'local_trials_idx_list': [0, 1, 2, 3, 4],
            'global_trials_idx_list': [20, 21, 22, 23, 24]
        },
        6: {
            'date': '20221027',
            'session_type': 'overt',
            'session': 'S03',
            'state': 6,
            'local_trials_idx_list': [0, 1, 2, 3, 4],
            'global_trials_idx_list': [25, 26, 27, 28, 29]
        },
        7: {
            'date': '20221027',
            'session_type': 'overt',
            'session': 'S03',
            'state': 7,
            'local_trials_idx_list': [0, 1, 2, 3, 4],
            'global_trials_idx_list': [30, 31, 32, 33, 34]
        },
        8: {
            'date': '20221027',
            'session_type': 'overt',
            'session': 'S03',
            'state': 8,
            'local_trials_idx_list': [0, 1, 2, 3, 4],
            'global_trials_idx_list': [35, 36, 37, 38, 39]
        }
    },
    'ccCenterOut_20221027_S04': {
        1: {
            'date': '20221027',
            'session_type': 'overt',
            'session': 'S04',
            'state': 1,
            'local_trials_idx_list': [0, 1, 2, 3, 4],
            'global_trials_idx_list': [40, 41, 42, 43, 44]
        },
        2: {
            'date': '20221027',
            'session_type': 'overt',
            'session': 'S04',
            'state': 2,
            'local_trials_idx_list': [0, 1, 2, 3, 4],
            'global_trials_idx_list': [45, 46, 47, 48, 49]
        },
        3: {
            'date': '20221027',
            'session_type': 'overt',
            'session': 'S04',
            'state': 3,
            'local_trials_idx_list': [0, 1, 2, 3, 4],
            'global_trials_idx_list': [50, 51, 52, 53, 54]
        },
        4: {
            'date': '20221027',
            'session_type': 'overt',
            'session': 'S04',
            'state': 4,
            'local_trials_idx_list': [0, 1, 2, 3, 4],
            'global_trials_idx_list': [55, 56, 57, 58, 59]
        },
        5: {
            'date': '20221027',
            'session_type': 'overt',
            'session': 'S04',
            'state': 5,
            'local_trials_idx_list': [0, 1, 2, 3, 4],
            'global_trials_idx_list': [60, 61, 62, 63, 64]
        },
        6: {
            'date': '20221027',
            'session_type': 'overt',
            'session': 'S04',
            'state': 6,
            'local_trials_idx_list': [0, 1, 2, 3, 4],
            'global_trials_idx_list': [65, 66, 67, 68, 69]
        },
        7: {
            'date': '20221027',
            'session_type': 'overt',
            'session': 'S04',
            'state': 7,
            'local_trials_idx_list': [0, 1, 2, 3, 4],
            'global_trials_idx_list': [70, 71, 72, 73, 74]
        },
        8: {
            'date': '

In [16]:
model_data_filenames_dict['ccCenterOut_20221027_S03']

{
    1: {
        'date': '20221027',
        'session_type': 'overt',
        'session': 'S03',
        'state': 1,
        'local_trials_idx_list': [0, 1, 2, 3, 4],
        'global_trials_idx_list': [0, 1, 2, 3, 4]
    },
    2: {
        'date': '20221027',
        'session_type': 'overt',
        'session': 'S03',
        'state': 2,
        'local_trials_idx_list': [0, 1, 2, 3, 4],
        'global_trials_idx_list': [5, 6, 7, 8, 9]
    },
    3: {
        'date': '20221027',
        'session_type': 'overt',
        'session': 'S03',
        'state': 3,
        'local_trials_idx_list': [0, 1, 2, 3, 4],
        'global_trials_idx_list': [10, 11, 12, 13, 14]
    },
    4: {
        'date': '20221027',
        'session_type': 'overt',
        'session': 'S03',
        'state': 4,
        'local_trials_idx_list': [0, 1, 2, 3, 4],
        'global_trials_idx_list': [15, 16, 17, 18, 19]
    },
    5: {
        'date': '20221027',
        'session_type': 'overt',
        'session': 'S03',
        'state': 5,
        'local_trials_idx_list': [0, 1, 2, 3, 4],
        'global_trials_idx_list': [20, 21, 22, 23, 24]
    },
    6: {
        'date': '20221027',
        'session_type': 'overt',
        'session': 'S03',
        'state': 6,
        'local_trials_idx_list': [0, 1, 2, 3, 4],
        'global_trials_idx_list': [25, 26, 27, 28, 29]
    },
    7: {
        'date': '20221027',
        'session_type': 'overt',
        'session': 'S03',
        'state': 7,
        'local_trials_idx_list': [0, 1, 2, 3, 4],
        'global_trials_idx_list': [30, 31, 32, 33, 34]
    },
    8: {
        'date': '20221027',
        'session_type': 'overt',
        'session': 'S03',
        'state': 8,
        'local_trials_idx_list': [0, 1, 2, 3, 4],
        'global_trials_idx_list': [35, 36, 37, 38, 39]
    }
}

In [6]:
trial_filenames_list = []
for partition_key, partition_data_func in model_data_filenames_dict.items():
    model_data_dict = partition_data_func #
    
    for state, trial_information_dict in model_data_dict.items():
        session_type = trial_information_dict['session_type']
        date = trial_information_dict['date']
        local_trials_idx_list = trial_information_dict['local_trials_idx_list']
        session = trial_information_dict['session']
        
        trial_filenames_list += [f"{session_type}_{date}_{session}_T{trial_idx}_state{state}" for trial_idx in local_trials_idx_list]
        
        
len(trial_filenames_list)

1120

In [105]:
data_split_type = model_data_params[current_experiment]['sel_split_type']

split_type_params = model_data_params[current_experiment]['split_types'][data_split_type]

leave_out = split_type_params['leave_out']
randomized = split_type_params['randomized']
random_seed = split_type_params['random_seed']
sel_session_type = model_data_params[current_experiment]['sel_session_type']

if data_split_type == 'leave_day_out':
    dates_list = []
    for partition_key, partition_data_func in model_data_filenames_dict.items():
        date = partition_key.split('_')[-2]
        session = partition_key.split('_')[-1]
        
        data_dict = partition_data_func #
        
        for state, trial_information_dict in data_dict.items():
            if sel_session_type != trial_information_dict['session_type']:
                break
            
            if date not in dates_list:
                dates_list.append(date)
                break
    
    dates_list = np.array(list(set(dates_list)))

    dates_list_permuted = np.random.permutation(dates_list)

    test_list_dates = dates_list_permuted[:leave_out]
    train_list_dates = dates_list_permuted[leave_out:]
    
    train_list = []
    test_list = []
    for partition_key, partition_data_func in model_data_filenames_dict.items():
        date = partition_key.split('_')[-2]
        session = partition_key.split('_')[-1]
        
        data_dict = partition_data_func #
        
        for state, trial_information_dict in data_dict.items():
            session_type = trial_information_dict['session_type']
            
            if sel_session_type != trial_information_dict['session_type']:
                continue
            
            if date in train_list_dates:
                train_list += trial_information_dict['global_trials_idx_list']
                
            elif date in test_list_dates:
                test_list += trial_information_dict['global_trials_idx_list']

elif data_split_type == 'leave_session_out':
    dates_and_sessions_list = []
    for partition_key, partition_data_func in model_data_filenames_dict.items():
        date = partition_key.split('_')[-2]
        session = partition_key.split('_')[-1]
        
        data_dict = partition_data_func #
        
        for state, trial_information_dict in data_dict.items():
            if sel_session_type != trial_information_dict['session_type']:
                break
            
            if {'date': date, 'session': session} not in dates_and_sessions_list:
                dates_and_sessions_list.append({'date': date, 'session': session})
                break

    dates_and_sessions_list = np.array(dates_and_sessions_list)
    dates_and_sessions_list_permuted = np.random.permutation(dates_and_sessions_list)
    
    test_list_dates_and_sessions = dates_and_sessions_list_permuted[:leave_out]
    train_list_dates_and_sessions = dates_and_sessions_list_permuted[leave_out:]
    
    train_list = []
    test_list = []
    for partition_key, partition_data_func in model_data_filenames_dict.items():
        date = partition_key.split('_')[-2]
        session = partition_key.split('_')[-1]
        
        data_dict = partition_data_func #
        for state, trial_information_dict in data_dict.items():
            if {'date': date, 'session': session} in train_list_dates_and_sessions:
                train_list += trial_information_dict['global_trials_idx_list']
                
            elif {'date': date, 'session': session} in test_list_dates_and_sessions:
                test_list += trial_information_dict['global_trials_idx_list']

elif data_split_type == 'leave_trial_out':
    trials_list = []
    for partition_key, partition_data_func in model_data_filenames_dict.items():
        date = partition_key.split('_')[-2]
        session = partition_key.split('_')[-1]
        
        data_dict = partition_data_func #
        
        for state, trial_information_dict in data_dict.items():
            if sel_session_type != trial_information_dict['session_type']:
                break
                
            trials_list += trial_information_dict['global_trials_idx_list']
            
    trials_list = np.array(trials_list)
    trials_list_permuted = np.random.permutation(trials_list)
    
    test_list = trials_list_permuted[:leave_out]
    train_list = trials_list_permuted[leave_out:]
    

array([533, 372, 570, 641, 740, 549, 335, 413, 139, 679, 263, 155, 524,
        39, 404, 357,  60, 480,  92, 459, 117, 526, 506,  49, 250, 354,
       422,  70, 735,  74, 790, 265, 511, 781, 700, 614,  16,  14, 532,
       361, 227, 465, 490, 300, 527, 187, 412, 162, 530, 124, 698, 371,
       200, 769,  25, 144, 794, 795, 203, 174, 113, 375, 623, 518,  35,
       284,  78,  61, 168, 708, 513,  65, 417, 547, 683, 315, 616, 311,
       109, 663, 380, 681, 656, 503, 105, 176, 373, 129, 164, 453, 171,
       468, 456, 712, 723, 166, 158, 564, 649, 255, 522, 296, 515, 279,
       474, 281, 211, 431, 792,   3, 235, 766, 674, 378,  76, 206, 320,
       334, 445, 560, 135, 388,   8, 326, 439, 793, 257, 687, 770, 342,
       495, 672, 653, 625, 327, 365, 535, 346, 267, 715,  83, 778, 230,
       757,  24, 134, 156, 722, 576, 389, 529, 544,  57, 799, 194, 787,
       578, 220, 593, 579, 487, 624, 108, 264, 566, 153, 457, 499, 182,
       479, 680,  40,  46,  55, 768, 119, 262, 772,  47, 362, 659, 352,
       786, 454, 534, 247, 771,  45, 442, 647, 691, 747, 761,  71, 764,
       626, 628, 266, 146, 238, 345, 231, 344, 317, 743, 132, 620, 172,
       478,  30, 398, 376,  29, 359, 407, 221,  90, 733, 660, 580, 444,
       699, 692, 774, 209, 133, 321, 393, 682, 582, 415, 193, 462, 258,
       676, 350, 337, 661, 738, 650, 443, 514,  36, 618, 360, 243, 348,
        98, 558, 782, 461, 592,  38, 312, 684, 395, 484, 347, 642, 539,
       120, 285, 491, 706, 621,  31, 161,  68, 610, 437, 210, 111, 424,
       713, 358, 688, 678, 241, 282, 195, 246,  95, 196, 671, 615, 545,
       410,  75, 229, 783, 644, 233, 784, 419, 762, 746, 475, 651, 249,
        10, 125, 760, 489, 319,  67, 137, 434, 216, 739, 632, 167,  17,
        18, 658, 179,  86, 779,  91, 608, 385, 386, 493, 669,  33,   6,
       314,  72, 537, 466, 274,  56, 648, 597, 114, 448, 185, 666,  64,
       540, 555, 485, 460,  34, 477, 212, 717, 538, 199, 112, 215,  79,
        81, 138, 636, 115, 298, 401,  28, 710, 575, 562,  53, 590, 438,
        51, 271, 744, 476, 720, 429, 634,   7, 791,  41, 510, 748, 605,
       305, 573, 356, 151, 277, 183, 332,  77,  82, 205, 637, 366, 728,
         5, 520, 287, 304, 299, 130, 339, 502,  27, 611, 541, 336, 777,
       561,  88, 574, 500, 675, 411,   1, 587, 107,  96, 214, 745, 652,
       213, 482, 705, 127, 548, 734, 496, 418, 701, 598, 471, 428, 338,
       703,  93, 225, 136,  58, 507,   9, 447,  59, 667, 552, 110, 508,
       256, 516, 236, 731, 736, 152, 208, 391, 630, 463, 409, 585,  52,
       118, 569, 184,  48, 613, 160, 370, 123, 292, 436, 308, 758, 451,
       154, 163, 286, 131, 796, 719, 584, 104, 289, 340, 432, 721, 693,
       423, 397, 517, 664, 201, 374,  13, 519,  19, 313, 694,  50, 726,
       633,  44, 599, 481, 261, 368, 494, 732, 775, 603, 655, 528, 291,
       207, 228,  11,  32, 306, 767, 143, 741, 686, 559, 310, 181, 381,
       631, 730, 567, 645, 309, 550, 294, 142, 543, 364, 668, 148, 218,
       202, 486, 677, 441, 302, 765, 763, 303, 240, 759, 497, 369, 269,
       101, 640, 406, 276, 301, 577, 392, 657, 643, 604,   0, 204, 355,
       483, 646, 307, 521, 288, 588, 239, 383, 750, 100, 222, 341, 469,
       217,  22, 617, 725, 586, 223, 387, 498, 159,  69,  87, 557,  99,
       253, 798, 149,  62, 399, 140,  12, 150, 173, 622, 260, 145, 283,
       553, 353, 742, 244, 390, 325, 245, 601, 729, 189,  97, 531,  89,
       492,  26, 103, 421,  85, 556, 711,  43, 714, 197, 591, 330, 690,
       408, 607, 612, 600, 696, 501, 414, 363, 450, 697, 695, 785, 316,
       425, 789,   4, 727, 751, 224, 753, 780, 343,  21, 670, 272, 654,
       324, 458, 295, 509, 405, 268, 384, 665,  15, 141, 689, 467,  23,
       328, 433, 116, 464, 420, 157,  54, 629, 776, 219, 470, 126,  37,
       542, 177, 416,  66, 226, 377,   2, 724, 242, 749, 606, 440, 702,
        20, 188, 572, 190, 400, 396, 270, 571, 251, 627, 191, 280, 638,
       297, 754, 619, 273, 237, 232, 198, 102, 427, 472, 512, 254, 455,
       563, 180, 716, 581, 252,  80, 752

In [107]:
len(test_list)

20

In [90]:
train_list_dates_and_sessions

array([{'date': '20221027', 'session': 'S08'},
       {'date': '20221104', 'session': 'S10'},
       {'date': '20221108', 'session': 'S09'}], dtype=object)

In [34]:
x = [{'a': 1, 'b': 2}, {'c': 1, 'd':2}]
{'a': 1, 'b': 3} in x

False

In [37]:
test_list

[
    600,
    601,
    602,
    603,
    604,
    605,
    606,
    607,
    608,
    609,
    610,
    611,
    612,
    613,
    614,
    615,
    616,
    617,
    618,
    619,
    620,
    621,
    622,
    623,
    624,
    625,
    626,
    627,
    628,
    629,
    630,
    631,
    632,
    633,
    634,
    635,
    636,
    637,
    638,
    639,
    640,
    641,
    642,
    643,
    644,
    645,
    646,
    647,
    648,
    649,
    650,
    651,
    652,
    653,
    654,
    655,
    656,
    657,
    658,
    659,
    660,
    661,
    662,
    663,
    664,
    665,
    666,
    667,
    668,
    669,
    670,
    671,
    672,
    673,
    674,
    675,
    676,
    677,
    678,
    679,
    680,
    681,
    682,
    683,
    684,
    685,
    686,
    687,
    688,
    689,
    690,
    691,
    692,
    693,
    694,
    695,
    696,
    697,
    698,
    699,
    700,
    701,
    702,
    703,
    704,
    705,
    706,
    707,
    708,
    709,
    710,
    711,
    712,
    713,
    714,
    715,
    716,
    717,
    718,
    719,
    720,
    721,
    722,
    723,
    724,
    725,
    726,
    727,
    728,
    729,
    730,
    731,
    732,
    733,
    734,
    735,
    736,
    737,
    738,
    739,
    740,
    741,
    742,
    743,
    744,
    745,
    746,
    747,
    748,
    749,
    750,
    751,
    752,
    753,
    754,
    755,
    756,
    757,
    758,
    759,
    760,
    761,
    762,
    763,
    764,
    765,
    766,
    767,
    768,
    769,
    770,
    771,
    772,
    773,
    774,
    775,
    776,
    777,
    778,
    779,
    780,
    781,
    782,
    783,
    784,
    785,
    786,
    787,
    788,
    789,
    790,
    791,
    792,
    793,
    794,
    795,
    796,
    797,
    798,
    799
]

In [32]:
print(len(test_list))
print(len(train_list))

120
680


In [35]:
print(len(test_list))
print(len(train_list))

200
600


In [42]:
model_data_dict

{
    'overt_20221027_S03_T0_state1': <function create_closure_func.<locals>.closure at 0x7fecf1237d00>,
    'overt_20221027_S03_T1_state1': <function create_closure_func.<locals>.closure at 0x7fecf152db40>,
    'overt_20221027_S03_T2_state1': <function create_closure_func.<locals>.closure at 0x7fecf152dab0>,
    'overt_20221027_S03_T3_state1': <function create_closure_func.<locals>.closure at 0x7fecf152c700>,
    'overt_20221027_S03_T4_state1': <function create_closure_func.<locals>.closure at 0x7fecf152c0d0>,
    'overt_20221027_S03_T0_state2': <function create_closure_func.<locals>.closure at 0x7fecf152dbd0>,
    'overt_20221027_S03_T1_state2': <function create_closure_func.<locals>.closure at 0x7fecf152d990>,
    'overt_20221027_S03_T2_state2': <function create_closure_func.<locals>.closure at 0x7fecf152da20>,
    'overt_20221027_S03_T3_state2': <function create_closure_func.<locals>.closure at 0x7fecf152d900>,
    'overt_20221027_S03_T4_state2': <function create_closure_func.<locals>.closure at 0x7fecf152dd80>,
    'overt_20221027_S03_T0_state3': <function create_closure_func.<locals>.closure at 0x7fecf152dea0>,
    'overt_20221027_S03_T1_state3': <function create_closure_func.<locals>.closure at 0x7fecf152d870>,
    'overt_20221027_S03_T2_state3': <function create_closure_func.<locals>.closure at 0x7fecf152d7e0>,
    'overt_20221027_S03_T3_state3': <function create_closure_func.<locals>.closure at 0x7fecf152d630>,
    'overt_20221027_S03_T4_state3': <function create_closure_func.<locals>.closure at 0x7fecf152c670>,
    'overt_20221027_S03_T0_state4': <function create_closure_func.<locals>.closure at 0x7fecf152d5a0>,
    'overt_20221027_S03_T1_state4': <function create_closure_func.<locals>.closure at 0x7fecf152caf0>,
    'overt_20221027_S03_T2_state4': <function create_closure_func.<locals>.closure at 0x7fecf152ce50>,
    'overt_20221027_S03_T3_state4': <function create_closure_func.<locals>.closure at 0x7fecf152c940>,
    'overt_20221027_S03_T4_state4': <function create_closure_func.<locals>.closure at 0x7fecf152c9d0>,
    'overt_20221027_S03_T0_state5': <function create_closure_func.<locals>.closure at 0x7fecf152cdc0>,
    'overt_20221027_S03_T1_state5': <function create_closure_func.<locals>.closure at 0x7fecf152c820>,
    'overt_20221027_S03_T2_state5': <function create_closure_func.<locals>.closure at 0x7fecf152c8b0>,
    'overt_20221027_S03_T3_state5': <function create_closure_func.<locals>.closure at 0x7fecf152c040>,
    'overt_20221027_S03_T4_state5': <function create_closure_func.<locals>.closure at 0x7fecf152ca60>,
    'overt_20221027_S03_T0_state6': <function create_closure_func.<locals>.closure at 0x7fecf152d6c0>,
    'overt_20221027_S03_T1_state6': <function create_closure_func.<locals>.closure at 0x7fecf152c160>,
    'overt_20221027_S03_T2_state6': <function create_closure_func.<locals>.closure at 0x7fecf152d000>,
    'overt_20221027_S03_T3_state6': <function create_closure_func.<locals>.closure at 0x7fecf152cb80>,
    'overt_20221027_S03_T4_state6': <function create_closure_func.<locals>.closure at 0x7fecf152c1f0>,
    'overt_20221027_S03_T0_state7': <function create_closure_func.<locals>.closure at 0x7fecf152c790>,
    'overt_20221027_S03_T1_state7': <function create_closure_func.<locals>.closure at 0x7fecf152c4c0>,
    'overt_20221027_S03_T2_state7': <function create_closure_func.<locals>.closure at 0x7fecf152c430>,
    'overt_20221027_S03_T3_state7': <function create_closure_func.<locals>.closure at 0x7fecf152cd30>,
    'overt_20221027_S03_T4_state7': <function create_closure_func.<locals>.closure at 0x7fecf152c310>,
    'overt_20221027_S03_T0_state8': <function create_closure_func.<locals>.closure at 0x7fecf152d2d0>,
    'overt_20221027_S03_T1_state8': <function create_closure_func.<locals>.closure at 0x7fecf152cca0>,
    'overt_20221027_S03_T2_state8': <function create_closure_func.<locals>.closure at 0x7fecf152d750>,
    'overt_20221027_S03_T3_state8': <function create_closure_func.<locals>.closure a

In [ ]:
train_dat.shape

In [ ]:
np.insert(total_cum_sum, 0, 0)

In [ ]:
total_len = np.sum([x.shape[0] for x in train_list])
total_cum_sum = np.cumsum([x.shape[0] for x in train_list])

train_dat = np.zeros((total_len, train_list[0].shape[1], train_list[0].shape[2], train_list[0].shape[3])).astype(np.int32)

total_cum_sum = np.insert(total_cum_sum, 0, 0)

for idx in range(1, len(total_cum_sum)):
    
    if idx != 0:
        continue
    
    start_idx = total_cum_sum[idx - 1]
    end_idx = total_cum_sum[idx]
    
    print(start_idx, end_idx)
    
    print(train_list[idx-1].dtype)
    
    # train_dat[start_idx:end_idx,:,:,:] = train_list[idx-1]


# np.concatenate(train_list).shape

In [15]:
model_data_dict['overt']['20221027'][1][3].shape

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1096040/1487801706.py:1 in <cell line: 1>                                         │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1096040/1487801706.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'overt'

In [24]:
np.random.permutation(np.array(list(set(['a', 'b', 'c', 'c']))))

array(['c', 'b', 'a'], dtype='<U1')